# Финальный проект: SQL

Цель исследования: анализ базы данных сервиса по читению книг

План работы: 
* Ислледовать таблицы, выгрузить первые строки.
* Сделать SQL-запросы для решения каждого задания:
    * Посчитать, сколько книг вышло после 1 января 2000 года;
    * Для каждой книги посчитать количество обзоров и среднюю оценку;
    * Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
    * Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
    * Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
* Вывести результаты каждого запроса.
* Описать выводы по каждой из решённых задач.

Описание данных:


Таблица books - содержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.


Таблица authors - cодержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.


Таблица publishers - cодержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;


Таблица ratings - cодержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.


Таблица reviews- cодержит данные о пользовательских обзорах:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.


In [15]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследуем таблицы, выгрузим первые строки:

### Таблица books:

In [16]:
query = '''
SELECT * 
FROM books 
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица authors:

In [7]:
# выгрузим первые строки таблицы authors:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.read_sql_query(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица publishers:

In [8]:
# выгрузим первые строки таблицы publishers:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.read_sql_query(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица ratings:

In [9]:
# выгрузим первые строки таблицы ratings:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.read_sql_query(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews:

In [11]:
# выгрузим первые строки таблицы reviews:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.read_sql_query(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## SQL-запросы

### Посчитаем, сколько книг вышло после 1 января 2000 года:

In [18]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01'
'''
pd.read_sql_query(query, con = engine) 

,count
0,821


#### Вывод:
после 1 января 2000 года вышла 821 книга.

### Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [22]:
query = '''
SELECT b.book_id,
    b. title,
    COUNT(r.review_id) AS count_reviews,
    AVG(rat.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC
'''
pd.read_sql_query(query, con = engine) 

,book_id,title,count_reviews,avg_rating
0,518,Pop Goes the Weasel (Alex Cross #5),4,5.00
1,732,The Ghost Map: The Story of London's Most Terr...,4,5.00
2,347,In the Hand of the Goddess (Song of the Liones...,6,5.00
3,610,Tai-Pan (Asian Saga #2),4,5.00
4,330,How to Be a Domestic Goddess: Baking and the A...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,12,2.25
996,371,Junky,4,2.00
997,316,His Excellency: George Washington,4,2.00
998,202,Drowning Ruth,9,2.00


#### Вывод:
Определили для каждой книги количество обзоров и среднюю оценку. Всего книг - 999, наивысшая оценка - 5.00, наименьшая оценка - 1.50 

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц:

In [27]:
query='''
SELECT p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS count_books
FROM publishers AS p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY count_books DESC
LIMIT 1'''

pd.read_sql_query(query, con = engine)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


#### Вывод:
Издательство, которое выпустило наибольшее число книг тольще 50 страниц - Penguin Books. Данное издательство выпустило 42 книги

### Определим автора с самой высокой средней оценкой книг:
Учтем только книги с 50 и более оценками

In [29]:
query = '''
SELECT a.author_id,
    a.author,
    AVG(r.rating) AS avg_rating,
    COUNT(r.rating_id) AS number_of_rating
FROM authors AS a
JOIN books AS b ON a.author_id = b.author_id
JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY a.author_id
HAVING COUNT(r.rating_id) >= 50
ORDER BY number_of_rating DESC
LIMIT 1
'''
pd.read_sql_query(query, con = engine)

,author_id,author,avg_rating,number_of_rating
0,236,J.K. Rowling/Mary GrandPré,4.288462,312


#### Вывод:
Автор с самой высокой средней оценкой книг - J.K. Rowling. Средний рейтинг - 4.3

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок:

In [35]:
query = '''
SELECT AVG(r.count_reviews) AS avg_reviews
FROM (SELECT COUNT(review_id) AS count_reviews
     FROM reviews 
     WHERE username IN (SELECT username
                       FROM ratings
                       GROUP BY username
                       HAVING COUNT(ratings) > 48)
     GROUP BY username) AS r
'''

pd.read_sql_query(query, con = engine)

,avg_reviews
0,24.0


#### Вывод:
среднее количество обзоров от пользователей, оставивших больше 48 оценок - 24

## Вывод:
* После 1 января 2000 года вышла 821 книга.
* Всего книг - 999, наивысшая оценка - 5.00, наименьшая оценка - 1.50.
* Издательство, которое выпустило наибольшее число книг тольще 50 страниц - Penguin Books. Данное издательство выпустило 42 книги.
* Автор с самой высокой средней оценкой книг - J.K. Rowling. Средний рейтинг - 4.3.
* Среднее количество обзоров от пользователей, оставивших больше 48 оценок - 24.